In [28]:
import os



def get_corpus(path = 'JOURNO/', prefix='jj'):
    fns = [path + x for x in os.listdir(path) if x[:2] == prefix]

    arts, bits = [], []
    for fn in fns[:]:
        with open(fn,'r') as fp:
            dd = [fp.readline().strip() for x in range(3)]
        bits.append((dd[0],dd[2]) )
        
        with open(fns[i]) as fp:
            dd = ''.join(fp.readlines()[4:-2]).replace('\n', ' ')

        arts.append(dd)
        
    return bits, arts

In [29]:
bits, arts = get_corpus()

I scraped articles to a txt file, the format is as follows

- line 0 = title
- blank
- line 2 = contents of the meta i.e. kwords
- blank
- contents
- blank
- line -1 = src URL



In [30]:
titles, kwords = zip(* bits)
vocab = ','.join(kwords).split(',')

In [31]:
kwords = list(set(','.join(kwords).split(',')))
kwords = sorted(kwords)

In [33]:
#kwords

Given the format one can get the contents as follows

In [13]:
arts  = []
bag = range(len(bits))
for i in bag:
    with open(fns[i],'r') as fp:
        dd = ''.join(fp.readlines()[4:-2]).replace('\n', ' ')
      
    arts.append(dd)

In [14]:

lengths = [len(x) for x in arts]
totals = [(sum(lengths[:i]),i)  for i,x in enumerate(lengths)]
totals[-1]

(2915338, 615)

In [100]:
tx = ' '.join(arts)
with open('text6.txt', 'w') as fp:
    fp.write(tx)

In [19]:
from collections import Counter

word_count = Counter(vocab)
wc = sorted(list(word_count.items()), key = lambda x : -x[1] )

In [20]:
wc

[('Fashion', 614),
 ('Life and style', 592),
 ('Women', 63),
 ("Men's fashion", 41),
 ('Celebrity', 39),
 ('Media', 33),
 ('Culture', 32),
 ('Beauty', 30),
 ('World news', 30),
 ("Women's shoes", 29),
 ('Film', 22),
 ('Music', 19),
 ('US news', 18),
 ('UK news', 17),
 ('Newspapers & magazines', 16),
 ('Magazines', 15),
 ('Politics', 15),
 ('Kate Moss', 13),
 ('Models', 13),
 ('Society', 13),
 ('Donald Trump', 12),
 ('Sport', 11),
 ('Health & wellbeing', 11),
 ('Jeans', 11),
 ('Body image', 10),
 ('Feminism', 10),
 ('Television', 10),
 ('Newspapers', 9),
 ('Christmas', 9),
 ('Sexual harassment', 9),
 ('Books', 8),
 ('Dresses', 8),
 ('Fashion industry', 8),
 ('Parents and parenting', 7),
 ('Weddings', 7),
 ('Duchess of Cambridge', 7),
 ('Fashion weeks', 7),
 ("Women's coats and jackets", 7),
 ('Tights and socks', 7),
 ('Glastonbury festival', 6),
 ('Makeup', 6),
 ('Kim Kardashian West', 6),
 ('Family', 6),
 ('Gender', 6),
 ('Business', 6),
 ('Lingerie', 6),
 ("Women's trousers", 6),
 ('T

In [54]:
junk = ','.join(word_count.keys() )

In [15]:
import textacy

In [16]:
doc = textacy.doc.Doc(junk)

NameError: name 'junk' is not defined

In [78]:
names = list(textacy.extract.named_entities(doc, drop_determiners=True, exclude_types='numeric'))

people = [x for x in names if x.label_ == 'PERSON' and len(x) > 1]
people[0].label_ == 'ORG'

False

In [73]:
people[0].text, len(people[0]

('Julian Assange', 2)

In [76]:
','.join([x.text for x in people if len(x) < 2])

'Models,Bestival,Photography,Cameron,Royal,Galloway,Men,Burberry,Cameron,Ant,Tights,Parks,Rosetta,Zara,Seinfeld,Older,Drama,Pornography,Rihanna,Royal,Biography,Burkini,Theresa,Depp,Slimane,Law'

In [77]:
others = 'Cameron,Royal,Galloway,Cameron,Seinfeld,Rihanna,Theresa,Depp,Slimane'

In [81]:
persons = [ x.text for x in people]
persons.extend(others.split(','))

In [82]:
bag = []

for i, kk in enumerate(kwords):
    for x in persons:
        if x in kk : 
            bag.append(i)
            break
            

['Seinfeld', 'Rihanna', 'Theresa', 'Depp', 'Slimane']

In [102]:
arts[0]

'I read somewhere that Blake Lively – who I guess is a person and not an exotic fruit – is to play a young Carrie Bradshaw. Didn\'t you say that the Sex and the City films had killed the franchise? How is this being allowed?  Mariella, I was as shocked as you. As you say, I made that ringing decree and read the last rites over the flailing corpse of Sex and the City and the series producer, Michael Patrick King, has wilfully – nay, foolishly – disregarded my command and is said to be planning a prequel to the series in which Ms Lively plays a young Carrie. Ah, prequels: sometimes good in theory, never good in practice. George Lucas, what are your thoughts on the matter?  While we await George\'s opinion, I need to stress two things: one, I personally think Blake Lively sounds more like the colour of a paint than an exotic fruit ("Yes, I like the idea of green for the kitchen but I don\'t want one that\'s too blake lively"). And two, this is a story that came from a terribly trashy maga

In [145]:
xx = textacy.preprocess_text(arts[0], lowercase=False, no_punct=True)
doc = textacy.doc.Doc(arts[0])

In [226]:
names = list(textacy.extract.named_entities(doc, drop_determiners=True, exclude_types='numeric'))
names = list(set(names))
names.sort(key = lambda x : -len(x))

nearly_unique = [ x.text for x in names if x.label_ == 'PERSON']
#monikers = [x for x in nearly_unique if ' ' not in x]
#set(nearly_unique)
nearly_unique = list(set(nearly_unique))
nearly_unique

['Sarah Jessica Parker',
 'Julian Assange',
 'King',
 'Gossip Girl',
 'George',
 'Kate Moss',
 'Mariella',
 'Carrie Bradshaw',
 "Osama bin Laden's",
 'Blake Lively – who',
 'Ding',
 'Philip',
 'Midas',
 'Michael Patrick King',
 'Andrea Dworkin-levels',
 'Jennifer Grey',
 'Carrie',
 'Blake Lively',
 'Tavi',
 'George Lucas']

# Replacement strategy

This is a greedy strategy.

1. try and replace the biggest string first
1. replace the substrings which should be forename, last name
1. finally look at what is left

In [201]:
replacers = []
for  x in nearly_unique:
    tt = x.split(' ')
    nn = len(tt) 
    if nn > 1:
        tt = x, x.split(' ')[0], x.split(' ')[-1]
    replacers.append(tt)
    
big_replacers = [x for x in replacers if len(x) > 1]
little_replacers = [x for x in replacers if len(x) == 1]

txt = arts[0]
for i, x in enumerate(big_replacers):
    for ss in x:
        txt = txt.replace(ss,'PERSON%d'%i)
        
total = i
for i, ss in enumerate(little_replacers):

    txt = txt.replace(ss[0],'PERSON%d'%(i+total) )


Now we'll deal with the other entities.

In [218]:
replacers2 = []
cats = ['WORK_OF_ART', 'ORG', 'GPE', 'PRODUCT']
tots = {x: 0 for x in cats}
for x in names:
    if x.label_ in cats:
        payload = x.text, '%s%d'%(x.label_, tots[x.label_])
        tots[x.label_] += 1
        replacers2.append(payload) 
        
for a,b in replacers2:
    txt = txt.replace(a,b )       



In [243]:
trash = []

for base_txt in arts[:]:

    doc = textacy.doc.Doc(base_txt)
    txt = base_txt[:]

    names = list(textacy.extract.named_entities(doc, drop_determiners=True, exclude_types='numeric'))
    names = list(set(names))
    names.sort(key = lambda x : -len(x))

    nearly_unique = [ x.text for x in names if x.label_ == 'PERSON']


    replacers = []
    for  x in list(set(nearly_unique)):
        tt = x.split(' ')
        nn = len(tt) 
        if nn > 1:
            tt = x, x.split(' ')[0], x.split(' ')[-1]
        replacers.append(tt)

    big_replacers = [x for x in replacers if len(x) > 1]
    little_replacers = [x for x in replacers if len(x) == 1]

    # for some reason there was 'in' in the repacement queue
    for i, x in enumerate(big_replacers):
        for ss in x:
            if len(ss) < 3: continue
            txt = txt.replace(ss,'PERSON%d'%i)

    total = i
    for i, ss in enumerate(little_replacers):
        if len(ss) < 3: continue
        txt = txt.replace(ss[0],'PERSON%d'%(i+total) )


    replacers2 = []
    cats = ['WORK_OF_ART', 'ORG', 'GPE', 'PRODUCT']
    tots = {x: 0 for x in cats}
    for x in names:
        #need to fix this
        if len(x.text) < 3: continue
        if x.label_ in cats:
            payload = x.text, '%s%d'%(x.label_, tots[x.label_])
            tots[x.label_] += 1
            replacers2.append(payload) 

    for a,b in replacers2:
        txt = txt.replace(a,b ) 
    
    trash.append(txt)
        
    

In [244]:
trash[-1]

'PERSON4 has protested against airbrushing by posing topless in a magazine. Is this really the best way to go about protesting?  For those who have missed this enormous international story of grave importance, PERSON4 agreed to pose topless for the US magazine ORG11 on the condition that her image would remain wholly unphotoshopped.  “I’ve had my body manipulated so many different times for so many different reasons, whether it’s paparazzi photographers or for film posters. And that [shoot] was one of the ones where I said: ‘OK, I’m fine doing the topless shot so long as you don’t make them any bigger or retouch.’ Because it does feel important to say it really doesn’t matter what shape you are,” PERSON4 told the ORG8.  And she’s right, of course. Few things matter less than what shape a person is. And yet, weirdly, few things are imbued with more significance by the media and society than a person’s physical shape, especially when that person is a woman. So I really, really want to ch

In [245]:
all_text = ' '.join(trash)

So finally I decided to escape the entity types using underscores.
- this may disturb the (relative) probability of getting an entity

In [255]:
import re

#sort by decreasing frequency

stuff = sorted( [ (-len(re.findall(x, all_text) ), x) for x in  ['PERSON',  'ORG','GPE','PRODUCT', 'WORK_OF_ART'] ] )
junk, kinds = zip(*stuff)
txt = all_text[:]
for i,x in enumerate(kinds):
    txt = txt.replace(x, '_'*(i+1))
    


In [258]:
with open('text7.txt', 'w') as fp:
    fp.write(txt.lower())

In [1]:
with open('text7.txt', 'r') as fp:
    data = fp.read()

In [2]:
data[:200]

"i read somewhere that _6 i guess is a person and not an exotic fruit – is to play a young _4. didn't you say that the ____0 had killed the franchise? how is this being allowed?  mariella, i was as sho"

In [22]:

#import spacy, and spacy french model
# spacy is used to work on text
import spacy
nlp = spacy.load('en')


In [25]:

with open('text6.txt', 'r') as fp:
    data = fp.read()

In [46]:
import re

word_list = []

stops = ("\n","\n\n", 'ask.hadley@guardian.co.uk')

for txt in arts[:5]:
    txt = re.sub("n\'t"," not",txt)
    txt = re.sub("I\'d","Id",txt)
    txt = re.sub("\'s"," is",txt)
    txt = re.sub("\'ve"," have",txt)
    doc = nlp(txt)
    txt_words = list(set([  w.text.lower() for w in doc if w.text not in stops ] ))
    word_list.extend(txt_words)
    
  

In [47]:
word_list

['bulk',
 'first',
 'forget',
 'royal',
 'properly',
 'wear',
 'they',
 'everyone',
 'careful',
 'ie',
 'else',
 '90',
 'prefer',
 'with',
 'pippa',
 'uncomfortable',
 'am',
 'individually',
 'issue',
 'ways',
 'detail',
 'already',
 'influenced',
 'designers',
 'link',
 'bunchy',
 'often',
 'bracelet',
 'york',
 'because',
 'awkwardly',
 'column',
 'lesser',
 'roksanda',
 'judge',
 'information',
 'cues',
 'through',
 'even',
 'unflattering',
 'wedding',
 'dresses',
 'commercial',
 'victoria',
 'black',
 'allow',
 'guardian',
 'backside',
 'my',
 'useless',
 'hand',
 'much',
 'prettiest',
 'frustratingly',
 'extensive',
 'at',
 'is',
 'mainly',
 'high',
 'way',
 'both',
 'mindset',
 'same',
 'above',
 'dressmaking',
 'short',
 'there',
 'manufacturer',
 'stupid',
 'powered',
 'needs',
 'hear',
 'quarters',
 'think',
 'or',
 'being',
 'tatty',
 'a',
 'needle',
 'also',
 'scholar',
 'post',
 'into',
 'that',
 'of',
 'best',
 'all',
 'fear',
 'added',
 'he',
 'suggest',
 'frame',
 'want'